<a href="https://colab.research.google.com/github/chungntu/1DCNN-LSTM-ResNet/blob/main/Problem_2_Clamped_bar_subjected_to_point_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

# =========================
# problem2.py (converted from problem2.m)
# =========================

def shape_function_L2(xi: float):
    # shape function and derivatives for L2 elements
    # xi in [-1, 1]
    shape = np.array([(1 - xi) / 2, (1 + xi) / 2], dtype=float)
    natural_derivatives = np.array([-0.5, 0.5], dtype=float)  # [-1; 1]/2
    return shape, natural_derivatives

def solution(GDof: int, prescribedDof, stiffness: np.ndarray, force: np.ndarray):
    # function to find solution in terms of global displacements
    activeDof = np.setdiff1d(np.arange(GDof), np.array(prescribedDof, dtype=int))
    U = np.linalg.solve(stiffness[np.ix_(activeDof, activeDof)], force[activeDof])
    displacements = np.zeros(GDof, dtype=float)
    displacements[activeDof] = U
    return displacements

def outputDisplacementsReactions(displacements, stiffness, GDof, prescribedDof):
    # output of displacements and reactions in tabular form

    # displacements
    print("Displacements")
    jj = np.arange(1, GDof + 1).reshape(-1, 1)  # MATLAB: jj=1:GDof
    disp_col = displacements.reshape(-1, 1)
    print(np.hstack([jj, disp_col]))

    # reactions
    F = stiffness @ displacements               # MATLAB: F=stiffness*displacements
    prescribed = np.array(prescribedDof, dtype=int)
    reactions = F[prescribed]                  # MATLAB: reactions=F(prescribedDof)
    print("reactions")
    prescribed_col = (prescribed + 1).reshape(-1, 1)  # show 1-based like MATLAB
    reactions_col = reactions.reshape(-1, 1)
    print(np.hstack([prescribed_col, reactions_col]))

# E: modulus of elasticity
# A: area of cross section
# L: length of bar
E = 30e6
A = 1.0
EA = E * A
L = 90.0

# generation of coordinates and connectivities
numberElements = 3

# generation equal spaced coordinates
nodeCoordinates = np.linspace(0.0, L, numberElements + 1)
xx = nodeCoordinates.copy()

# numberNodes: number of nodes
numberNodes = nodeCoordinates.size

# elementNodes: connections at elements
ii = np.arange(1, numberElements + 1)  # MATLAB: ii=1:numberElements
# Convert to 0-based indices for Python assembly
elementNodes = np.column_stack([ii - 1, ii])

# for structure:
displacements = np.zeros(numberNodes, dtype=float)
force = np.zeros(numberNodes, dtype=float)
stiffness = np.zeros((numberNodes, numberNodes), dtype=float)

# applied load at node 2 (MATLAB force(2)=3000.0)
force[1] = 3000.0  # Python index 1 == node 2

# computation of the system stiffness matrix
for e in range(numberElements):
    # elementDof: element degrees of freedom (Dof)
    elementDof = elementNodes[e, :]     # two node indices (0-based)
    nn = len(elementDof)

    length_element = nodeCoordinates[elementDof[1]] - nodeCoordinates[elementDof[0]]
    detJacobian = length_element / 2.0
    invJacobian = 1.0 / detJacobian

    # central Gauss point (xi=0, weight W=2)
    shape, naturalDerivatives = shape_function_L2(0.0)
    Xderivatives = naturalDerivatives * invJacobian

    # B matrix
    B = np.zeros((1, nn), dtype=float)
    B[0, :nn] = Xderivatives[:]

    # stiffness assembly: K(edof,edof) += B'*B*2*detJ*EA
    stiffness[np.ix_(elementDof, elementDof)] += (B.T @ B) * (2.0 * detJacobian * EA)

# boundary conditions and solution
# prescribed dofs: fixed at x=min and x=max
fixedDof = np.where((xx == xx.min()) | (xx == xx.max()))[0]
prescribedDof = fixedDof.tolist()

# solution
GDof = numberNodes
displacements = solution(GDof, prescribedDof, stiffness, force)

# output displacements/reactions (MATLAB style)
outputDisplacementsReactions(displacements, stiffness, GDof, prescribedDof)


Displacements
[[1.e+00 0.e+00]
 [2.e+00 2.e-03]
 [3.e+00 1.e-03]
 [4.e+00 0.e+00]]
reactions
[[ 1.e+00 -2.e+03]
 [ 4.e+00 -1.e+03]]
